In [11]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime

region = 'euw1'

lol = LolWatcher(key)

In [16]:
players = pd.read_csv('players.txt')
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})

In [17]:
for i in range(players.shape[0]):
    region = players.loc[i,'region']
    pseudo = players.loc[i,'pseudo']
    puuid = lol.summoner.by_name(region,pseudo)['puuid']

    matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
    i = 100
    while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
        matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
        i = i + 100
    
    for j,i in enumerate(matchs):
        p = lol.match.by_id(region,i)
        for player in p['info']['participants']:
            if player['puuid'] == puuid:
                cName = player['championName']
                cId = player['championId']
                ePing = player['enemyMissingPings']
                pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + player['enemyMissingPings'] + player['enemyVisionPings']
        matchData.loc[j,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]
        if j == 50:
            continue

KeyError: 'enemyMissingPings'

In [ ]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6367571487,1857.0,1.681925e+12,1.681927e+12,Braum,201.0,10.0,0.0
1,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6365910916,1178.0,1.681822e+12,1.681824e+12,Kayn,141.0,0.0,0.0
2,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6365174837,1035.0,1.681760e+12,1.681761e+12,Lulu,117.0,1.0,0.0
3,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6365123438,994.0,1.681759e+12,1.681760e+12,Braum,201.0,3.0,0.0
4,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6363787945,1707.0,1.681672e+12,1.681674e+12,Karma,43.0,12.0,4.0
5,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6363740926,915.0,1.681670e+12,1.681671e+12,Nautilus,111.0,9.0,0.0
6,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6363690986,1416.0,1.681669e+12,1.681670e+12,Braum,201.0,3.0,1.0
7,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6363576289,1317.0,1.681665e+12,1.681666e+12,Sylas,517.0,1.0,0.0
8,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6359215935,1627.0,1.681410e+12,1.681412e+12,Nautilus,111.0,7.0,1.0
9,RKmBWrKuzlo4ZGADQc6zoxtRBRdPakcXDN6cipGyK1EGxI...,EUW1_6359182676,998.0,1.681409e+12,1.681410e+12,Braum,201.0,5.0,1.0
